# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas

import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [71]:
# Your code here:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')
print(salaries)

                        Name                              Job Titles  \
0          AARON,  JEFFERY M                                SERGEANT   
1            AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2        AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3        ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4          ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   
...                      ...                                     ...   
33178  ZYLINSKA,  KATARZYNA                           POLICE OFFICER   
33179     ZYMANTAS,  LAURA C                          POLICE OFFICER   
33180      ZYMANTAS,  MARK E                          POLICE OFFICER   
33181    ZYRKOWSKI,  CARLO E                          POLICE OFFICER   
33182   ZYSKOWSKI,  DARIUSZ                  CHIEF DATA BASE ANALYST   

             Department Full or Part-Time Salary or Hourly  Typical Hours  \
0                POLICE                 F           Salary

Examine the `salaries` dataset using the `head` function below.

In [7]:
# Your code here:

salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [9]:
salaries.isnull().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

In [14]:
salaries_df = pd.DataFrame(salaries)
salaries_df.value_counts()

Series([], dtype: int64)

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [35]:
salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [72]:
salaries['Department'].value_counts()


POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [40]:
import scipy.stats as st

In [70]:
# 1: Set Hypothesis

# H0: mu of hourly wage of all hourly workers = $30/h
# H1: mu of hourly wage of all hourly workers ≠ $30/h
  
  # two tailed test


In [ ]:
# 2: Significance Level
alpha = 0.05

In [73]:
# 3: Sample
salaries_sam = salaries[salaries['Salary or Hourly'] == 'Hourly']
salaries_sam

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
10,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,NaN,2.65
18,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
...,...,...,...,...,...,...,...,...
33164,"ZUREK, FRANCIS",ELECTRICAL MECHANIC,OEMC,F,Hourly,40.0,NaN,46.10
33168,"ZWARYCZ MANN, IRENE A",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
33169,"ZWARYCZ, THOMAS J",POOL MOTOR TRUCK DRIVER,WATER MGMNT,F,Hourly,40.0,NaN,35.60
33174,"ZYGADLO, JOHN P",MACHINIST (AUTOMOTIVE),GENERAL SERVICES,F,Hourly,40.0,NaN,46.35


In [74]:
# mean of the sample 

X_hat = salaries_sam['Hourly Rate'].mean()
X_hat

32.78855771628128

In [76]:
# 4. Compute the test statistic
# p-value


stat, p_value = st.ttest_1samp(salaries_sam['Hourly Rate'], 30)

print(stat)
print(p_value)

20.6198057854942
4.3230240486229894e-92


In [77]:
# DECIDE

p_value < alpha

# We reject H0

True

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [ ]:
# SET THE HYPOTESIS 



# H0: mu the annual salaries this year are lower or equal to last year's mean of USD 86000
# H1: mu the annual salaries this year are higher that last year's mean of USD 86000

# one tailed test


In [ ]:
# Significance level

alpha = 0.05

In [84]:
condition = (salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary') 

In [82]:
salaries_police = salaries[condition]
salaries_police

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
11,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,NaN,84054.0,NaN
12,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
...,...,...,...,...,...,...,...,...
33177,"ZYGMUNT, DAWID",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


In [89]:
# mean of the sample 

X_hat = salaries_police['Annual Salary'].mean()
X_hat

86486.41450313339

In [92]:
# Compute Statistics and p-value

stat, p_value = st.ttest_1samp(salaries_police['Annual Salary'], 86000.0, alternative='less')

print(stat)
print(p_value)


3.081997005712994
0.9989698298224517


In [93]:
# Decision 

p_value < alpha

# The H0 is not rejected because alpha is smaller than the p-value

False

Using the `crosstab` function, find the department that has the most hourly workers. 

In [106]:
hourly_workers = pd.DataFrame(salaries, columns = ['Department', 'Salary or Hourly'])


In [108]:
hourly_workers_cross = pd.crosstab(hourly_workers['Department'], hourly_workers['Salary or Hourly'], rownames=['Department'], colnames=['Salary or Hourly'])
hourly_workers_cross

Salary or Hourly,Hourly,Salary
Department,,
ADMIN HEARNG,0,39
ANIMAL CONTRL,19,62
AVIATION,1082,547
BOARD OF ELECTION,0,107
BOARD OF ETHICS,0,8
BUDGET & MGMT,2,44
BUILDINGS,0,269
BUSINESS AFFAIRS,7,164
CITY CLERK,0,84


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [109]:
# Set Hypothesis

# H0: mu the hourly wage is > than or equal to USD 35/hour
# H1: mu the hourly wage is < than USD 35/hour

# one tailed test

In [110]:
# Significance Level 

alpha = 0.05

In [115]:
# Sample

condition2 = hourly_workers_cross['Hourly'] != 0

hourly_workers_sample = hourly_workers_cross[condition2]

hourly_workers_sample

Salary or Hourly,Hourly,Salary
Department,,
ANIMAL CONTRL,19,62
AVIATION,1082,547
BUDGET & MGMT,2,44
BUSINESS AFFAIRS,7,164
CITY COUNCIL,64,347
COMMUNITY DEVELOPMENT,4,203
CULTURAL AFFAIRS,7,58
FAMILY & SUPPORT,287,328
FINANCE,44,516


In [117]:
X_hat = hourly_workers_sample['Hourly'].mean() 
X_hat


364.6363636363636

In [119]:
# Compute the statistics and p-value

stat, p_value = st.ttest_1samp(hourly_workers_sample['Hourly'], 35, alternative='less')

print(stat)
print(p_value)


2.6947221653296625
0.993216162437707


In [120]:
# Decision

p_value < alpha

# we do not reject H0 because the p_value is bigger than the alpha


False

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [124]:
hourly_workers_sample.head(5)

Salary or Hourly,Hourly,Salary
Department,,
ANIMAL CONTRL,19,62
AVIATION,1082,547
BUDGET & MGMT,2,44
BUSINESS AFFAIRS,7,164
CITY COUNCIL,64,347


In [126]:
X_hat = hourly_workers_sample['Hourly'].mean() 
X_hat

364.6363636363636

In [131]:
# Condidence of 95%

c = 0.95

In [132]:
sigma = hourly_workers_sample['Hourly'].std(ddof=1) 
sigma


573.7628965625623

In [133]:
n = len(hourly_workers_sample)
n

22

In [134]:
st.t.interval(c, n-1, loc=X_hat, scale = s/np.sqrt(n))


(359.265949767031, 370.0067775056963)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [136]:
condition = (salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary') & (salaries['Annual Salary'])
condition

0         True
1         True
2        False
3        False
4        False
         ...  
33178     True
33179     True
33180     True
33181     True
33182    False
Length: 33183, dtype: bool

In [137]:
police_salary = salaries[condition]

In [139]:
X_hat = police_salary['Annual Salary'].mean() 
X_hat

86486.41450313339

In [140]:
c = 0.95

In [141]:
# Standard Deviation 

sigma = police_salary['Annual Salary'].std(ddof=1) 
sigma

18272.228293992157

In [143]:
n = len(police_salary)
n

13404

In [145]:
st.t.interval(c, n-1, loc=X_hat, scale = s/np.sqrt(n))


(86486.20943107129, 86486.6195751955)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

